In [ ]:
!pip install -q transformers datasets peft bitsandbytes trl

# Headers

In [ ]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

import spacy
nlp = spacy.load('en_core_web_sm')

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Utils


In [ ]:
def concat_input_output(input, output):
  """It concatenates the input and the LLM output"""
  text = []
  for i,o in zip(input, output):
    text.append(f'{i}\n  {o}')
  return text

In [ ]:
def get_ner(text):
  """It returns the entities from a text"""
  doc = nlp(text)
  entities = ''
  for ent in doc.ents:
    entities += ent.text + ' '
  return entities

# Dataset

In [ ]:
retain_train_df = pd.read_parquet('/content/drive/MyDrive/GIL/Unlearning/data/retain_validation-00000-of-00001.parquet', engine='pyarrow')
retain_train_df = retain_train_df[['id','input', 'output']]
retain_train_df['text'] = concat_input_output(retain_train_df.input.values, retain_train_df.output.values)
retain_train_df['entities'] = retain_train_df.text.apply(get_ner)

forget_train_df = pd.read_parquet('/content/drive/MyDrive/GIL/Unlearning/data/forget_validation-00000-of-00001.parquet', engine='pyarrow')
forget_train_df = forget_train_df[['id','input', 'output']]
forget_train_df['text'] = concat_input_output(forget_train_df.input.values, forget_train_df.output.values)
forget_train_df['entities'] = forget_train_df.text.apply(get_ner)


In [ ]:
for i in range(3):
  print(retain_train_df.iloc[i].text)
  print(retain_train_df.iloc[i].entities)
  print('-----------')

Christopher J. Hardy

Christopher J. Hardy (born 1955) is an American physicist and inventor of several magnetic resonance imaging (MRI) subsystem technologies for use in real time MRI and cardiac MR imaging and spectroscopy.
Biography.
Hardy obtained his Ph.D. from the University of Illinois at Urbana-Champaign in March, 1983. He is currently a principal scientist and Coolidge Fellow at General Electric. He developed the first graphical approach that allowed physicians to explore anatomy in real time during cardiac MRI, as opposed to viewing groups of images at a later time, and he also developed a technique that improved imaging speed. Both accomplishments have gained widespread use. He has also
  led the teams that developed 32 channel and 128 channel General Electric MRI systems.
Hardy has written 98 research papers and 54 patents.
Christopher J. Hardy Christopher J. Hardy 1955 American Hardy Ph.D. the University of Illinois Urbana-Champaign March, 1983 Coolidge Fellow General Elec

# Load model base

In [ ]:
# LORA
LORA_R=8                         # lora_r
LORA_ALPHA=32                    # lora_alpha
LORA_DROPOUT=0.0                 # lora_dropout
LORA_TARGET_MODULES="q_proj,k_proj,q_attn,v_proj,o_proj"    # lora_target_modules


quantizationConfig = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

olmo = AutoModelForCausalLM.from_pretrained("allenai/OLMo-1B-0724-hf", quantization_config=quantizationConfig)
olmo = prepare_model_for_kbit_training(olmo)


# Set up lora
peft_config = LoraConfig(
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    r=LORA_R,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=LORA_TARGET_MODULES.split(","),
)

olmo = get_peft_model(olmo, peft_config)
olmo.print_trainable_parameters()


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.71G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

trainable params: 2,097,152 || all params: 1,281,884,160 || trainable%: 0.1636


# Finetune model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("allenai/OLMo-1B-0724-hf")
# Use '[PAD]' or a similar valid token as the padding token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
dataset = Dataset.from_pandas(forget_train_df)

training_args = SFTConfig(
    max_seq_length=512,
    report_to='none',
    output_dir="/tmp",
    dataset_text_field="entities",
    packing=True,
)

trainer = SFTTrainer(
        model=olmo,
        train_dataset=dataset,
        args=training_args,
        tokenizer=tokenizer,
    )

trainer.train()

olmo_finetuned = trainer.model

tokenizer_config.json:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


# Task Vector

In [ ]:
class TaskVector():
    def __init__(self, pretrained_model=None, finetuned_model=None, vector=None):
        """Initializes the task vector from a pretrained and a finetuned checkpoints.

        This can either be done by passing two state dicts (one corresponding to the
        pretrained model, and another to the finetuned model), or by directly passying in
        the task vector state dict.
        """
        if vector is not None:
            self.vector = vector
        else:
            assert pretrained_model is not None and finetuned_model is not None
            with torch.no_grad():
                pretrained_state_dict = pretrained_model.state_dict()
                finetuned_state_dict = finetuned_model.state_dict()
                self.vector = {}
                for key in pretrained_state_dict:
                    if pretrained_state_dict[key].dtype in [torch.int64, torch.uint8]:
                        continue
                    self.vector[key] = finetuned_state_dict[key] - pretrained_state_dict[key]

    def __add__(self, other):
        """Add two task vectors together."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                if key not in other.vector:
                    print(f'Warning, key {key} is not present in both task vectors.')
                    continue
                new_vector[key] = self.vector[key] + other.vector[key]
        return TaskVector(vector=new_vector)

    def __radd__(self, other):
        if other is None or isinstance(other, int):
            return self
        return self.__add__(other)

    def __neg__(self):
        """Negate a task vector."""
        with torch.no_grad():
            new_vector = {}
            for key in self.vector:
                new_vector[key] = - self.vector[key]
        return TaskVector(vector=new_vector)

    def apply_to(self, pre_model, scaling_coef=1.0):
        """Apply a task vector to a pretrained model."""
        with torch.no_grad():
            pretrained_model = pre_model
            new_state_dict = {}
            pretrained_state_dict = pretrained_model.state_dict()
            for key in pretrained_state_dict:
                if key not in self.vector:
                    print(f'Warning: key {key} is present in the pretrained state dict but not in the task vector')
                    continue
                new_state_dict[key] = pretrained_state_dict[key] + scaling_coef * self.vector[key]
        pretrained_model.load_state_dict(new_state_dict, strict=False)
        return pretrained_model


In [ ]:
# Create the task vector
task_vector = TaskVector(olmo, olmo_finetuned)
# Negate the task vector
neg_task_vector = -task_vector
# Apply the task vector
result_model = neg_task_vector.apply_to(olmo, scaling_coef=0.5)

In [ ]:
result_model.base_model.save_pretrained("/content/drive/MyDrive/GIL/Unlearning/result_model_TaskVectorNER")